In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *

In [0]:
 # Read Bronze Patient Table
bronze = spark.table("bronze_patient_dtable")
patient_silver = (
    bronze
    # Clean FIRST & LAST name (remove digits)
    .withColumn("FIRST", regexp_replace(col("FIRST"), "[0-9]", ""))
    .withColumn("LAST",  regexp_replace(col("LAST"),  "[0-9]", ""))
    # Full Name
    .withColumn("FULL_NAME", concat_ws(" ", "FIRST", "LAST"))
    # Replace NULL reason description
    .withColumn(
        "REASONDESCRIPTION",
        when(col("REASONDESCRIPTION").isNull(), "Not Specified")
        .otherwise(col("REASONDESCRIPTION"))
    )
    # Primary Key (Business Key)
    .withColumn("PRIMARY_KEY", md5(col("PATIENTID")))
    # Checksum – SCD2 change detection
    .withColumn(
        "CHK_SUM_TXT",
        md5(
            concat_ws(
                "||",
                col("FIRST"),
                col("BIRTHDATE"),
                col("DEATHDATE"),
                col("LAST"),
                col("MARITAL"),
                col("GENDER"),
                col("ADDRESS"),
                col("CITY"),
                col("STATE"),
                col("COUNTRY"),
                col("HOSPITALNAME"),
                col("REASONDESCRIPTION"),
                col("PAYERID")
            )
        )
    )
    # Correct Transaction code
    .withColumn("TransactionCode", lit("I"))          # Correct type: STRING
 
    # Current Record Indicator (use 1/0, INTEGER)
    .withColumn("CURR_RCD_IND", lit(1).cast("int"))   # 1 = current
 
    # Activation Status
    .withColumn(
        "ActivationInd",
        when(col("DEATHDATE").isNotNull(), "Inactive")
        .otherwise("Active")
    )  # STRING
 
    #Row Created timestamp
    .withColumn("CRET_TS", current_timestamp())
 
    #Row Update timestamp (9999-12-31 as timestamp)
    .withColumn("UPDT_TS", lit("9999-12-31").cast("timestamp"))
)

**DEDUPLICATION OF PATIENTS RECORDS**

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number

business_cols = [
    "PATIENTID","BIRTHDATE","DEATHDATE","FIRST","LAST","MARITAL","GENDER",
    "BIRTHPLACE","ADDRESS","CITY","STATE","COUNTRY","HOSPITALNAME","PAYERID",
    "REASONDESCRIPTION"
]

w = Window.partitionBy(business_cols).orderBy(col("LOAD_CTL_KEY").desc())

patient_sliver_dedup = patient_silver\
    .withColumn("rn", row_number().over(w)) \
    .filter("rn = 1") \
    .drop("rn")



In [0]:
display(patient_sliver_dedup)

In [0]:
patient_sliver_dedup.write.mode("overwrite").saveAsTable("silver_patient_dtable")

In [0]:
%sql
select * from silver_patient_dtable

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS gold_patient_scd2_dtable (
    PATIENTID STRING,
    FIRST STRING,
    LAST STRING,
    FULL_NAME STRING,
    BIRTHDATE  DATE,
    DEATHDATE DATE,
    MARITAL STRING,
    GENDER STRING,
    ADDRESS STRING,
    CITY STRING,
    STATE STRING,
    COUNTRY STRING,
    HOSPITAL_NAME STRING,
    REASONDESCRIPTION STRING,
    PAYERID STRING,
    PRIMARY_KEY STRING,
    CHK_SUM_TXT STRING,
    TransactionCode STRING,
    CURR_RCD_IND INT,
    ActivationInd STRING,
    CRET_TS TIMESTAMP,
    UPDT_TS TIMESTAMP,
    LOAD_CTL_KEY TIMESTAMP
    
  
)
USING delta
""")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *
bronze = spark.table("bronze_payer_dtable")
payer_silver = (
    bronze
    .withColumn("PHONE", when(col("PHONE").isNull(), "000-000-0000")
                          .otherwise(col("PHONE")))
    .withColumn("ZIP", when(col("ZIP").isNull(), "00000")
                        .otherwise(col("ZIP")))
    .withColumn("ADDRESS", when(col("ADDRESS").isNull(), "NOT SPECIFIED")
                        .otherwise(col("ADDRESS")))
    .withColumn("STATE_HEADQUARTERED", when(col("STATE_HEADQUARTERED").isNull(), "NOT SPECIFIED").otherwise(col("STATE_HEADQUARTERED")))
    .withColumn("CITY",when(col("CITY").isNull(), "NOT SPECIFIED").otherwise(col("CITY")))
    # BUSINESS KEYS
    .withColumn("PRIMARY_KEY", md5(col("PAYERID")))
    .withColumn("CHK_SUM_TXT",
        md5(concat_ws("||",
            col("NAME"),
            col("ADDRESS"),
            col("CITY"),
            col("STATE_HEADQUARTERED"),
            col("ZIP"),
            col("PHONE")
        ))
    )

    # SCD2 METADATA
    .withColumn("TransactionCode", lit("I"))
    #.withColumn("Start_Date", current_timestamp())
    #.withColumn("End_Date", lit("9999-12-31").cast("timestamp"))
    .withColumn("CURR_RCD_IND", lit(1).cast("int")) 
    .withColumn(
    "ActivationInd",
    when(col("NAME") == "NO_INSURANCE", "Inactive")
    .otherwise("Active"))
    .withColumn("CRET_TS", current_timestamp())
    .withColumn("UPDT_TS", lit("9999-12-31").cast("timestamp"))
)






**DEDUPLICATION OF PAYERS RECORDS**

In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number

business_cols = [
    "PAYERID","NAME","ADDRESS","CITY","STATE_HEADQUARTERED","ZIP","PHONE"
]

w = Window.partitionBy(business_cols).orderBy(col("LOAD_CTL_KEY").desc())

payer_sliver_dedup = payer_silver\
    .withColumn("rn", row_number().over(w)) \
    .filter("rn = 1") \
    .drop("rn")


In [0]:
payer_sliver_dedup.write.format("delta").mode("overwrite").saveAsTable("silver_payer_dtable")

In [0]:
%sql

select * from workspace.default.silver_payer_dtable

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS gold_payer_scd2_dtable (
    PAYERID STRING,
    NAME STRING,
    ADDRESS STRING,
    CITY STRING,
    STATE_HEADQUARTERED STRING,
    ZIP STRING,
    PHONE STRING,
    PRIMARY_KEY STRING,
    CHK_SUM_TXT STRING,
    TransactionCode STRING,
    CURR_RCD_IND int,
    ActivationInd STRING,
    CRET_TS TIMESTAMP,
    UPDT_TS TIMESTAMP,
    LOAD_CTL_KEY TIMESTAMP
    
)
USING delta
""")


In [0]:
%sql
--Drop table if exists gold_patient_scd2_dtable;
-- Drop table if exists gold_payer_scd2_dtable

In [0]:
%sql
--drop table silver_patient_dtable
--drop table silver_payer_dtable